In [61]:
import pandas as pd #tablas https://pandas.pydata.org/docs/user_guide/pyarrow.html
import numpy as np #estadistica https://numpy.org/doc/1.25/user/index.html#user
import matplotlib.pyplot as plt #gráficos https://matplotlib.org/stable/plot_types/index.html
import seaborn as sns #gráficos https://seaborn.pydata.org/tutorial/regression.html
from sklearn import linear_model #modelo de regresion https://scikit-learn.org/stable/
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from datetime import datetime #https://docs.python.org/3/library/datetime.html
from datetime import timedelta #this is a good time to learn about dates and timestamps. You can find some info here:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import ttest_1samp
import scipy.stats as st
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions
import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine  # for establishing the connection and authentication
from getpass import getpass  # To get the password without showing the input

At this point, we have created a model to predict who will make a donation and who won't (Classification Model). But, what about the ammount of money that each person will give?

In this lab, subset those that have made a donation (Target B) and use that subset to create a model to predict how much money will they give (Target D) (Regression Model).

BRINGING DATA (ONLY BRINGING THE 10 BEST COLUMNS ACCORDING TO RFE) FILTERING FOR TARGET_B == 1

In [62]:
data = pd.read_csv('filtered_data_raw.csv')
data.head()

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,TARGET_D,TARGET_B
0,11,5,10.0,7.741935,4,89,1,92,0.0,0
1,16,3,25.0,15.666667,2,94,1,93,0.0,0
2,14,5,5.0,7.481481,4,90,1,91,0.0,0
3,17,3,10.0,6.812500,4,87,1,87,0.0,0
4,15,17,15.0,6.864865,2,86,1,93,0.0,0


In [63]:
data_b = data.loc[data['TARGET_B'] == 1].reset_index(drop=True)
data_b

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,TARGET_D,TARGET_B
0,13,7,5.0,4.066667,3,87,1,88,4.0,1
1,21,3,7.0,6.181818,3,90,1,90,7.0,1
2,13,2,5.0,4.857143,3,86,1,93,5.0,1
3,15,2,10.0,11.000000,2,90,1,90,13.0,1
4,13,16,12.0,9.400000,3,92,1,92,10.0,1
...,...,...,...,...,...,...,...,...,...,...
4838,19,9,17.0,7.935667,1,86,1,89,20.0,1
4839,20,5,15.0,11.666667,1,94,1,93,15.0,1
4840,13,2,20.0,14.400000,3,86,1,89,3.0,1
4841,17,4,20.0,11.583333,1,90,1,90,10.0,1


- Use this new dataframe to create a model to predict how much they will donate (Target D)

In [64]:
y=  data_b['TARGET_D']
X = data_b.drop(['TARGET_D'], axis = 1) 

In [65]:
from sklearn.model_selection import cross_val_score

In [66]:
from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor()

from sklearn.preprocessing import StandardScaler
import numpy as np

transformer = StandardScaler()
transformer = transformer.fit(X)
X = transformer.transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

model_pipeline = [model1, model2, model3]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

{'Decision Tree Regressor': -0.01302200710888941, 'Linear Regression': 0.2692305838574979, 'KNN': 0.41957462902592013}


- Using the regression model, make predictions on all of the people our classification model predicted will donate.

#GETTING THE ARRAY WITH PREDICTIONS FOR ALL THE DATA FROM RANDOM_FOREST NOTEBOOK, USING THE MODEL WITH ONLY 10 COLUMNS

X1_B = X1.drop(['TARGET_D'], axis = 1) 
predictions_target_b_all = clf1.predict(X1_B)
len(predictions_target_b_all)
predicted_b = pd.DataFrame(predictions_target_b_all, columns=['PREDICTED_B'])
predicted_b.to_csv("PREDICTED_B", index=False)


In [88]:
predicted_b = pd.read_csv('PREDICTED_B.csv')
predicted_b.head()

,PREDICTED_B
0,1
1,0
2,1
3,1
4,0


In [68]:
data_predicted_b = pd.concat((data,predicted_b), axis=1)
data_predicted_b

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,TARGET_D,TARGET_B,PREDICTED_B
0,11,5,10.0,7.741935,4,89,1,92,0.0,0,1
1,16,3,25.0,15.666667,2,94,1,93,0.0,0,0
2,14,5,5.0,7.481481,4,90,1,91,0.0,0,1
3,17,3,10.0,6.812500,4,87,1,87,0.0,0,1
4,15,17,15.0,6.864865,2,86,1,93,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
95407,18,10,25.0,25.000000,1,96,1,96,0.0,0,0
95408,16,1,20.0,20.000000,1,96,1,96,0.0,0,0
95409,17,14,10.0,8.285714,3,95,1,96,0.0,0,1
95410,16,6,18.0,12.146341,4,86,1,90,18.0,1,1


In [69]:
predicting_d = data_predicted_b.drop(['TARGET_D', 'TARGET_B'], axis = 1) 

In [70]:
predicting_d['TARGET_B'] = predicting_d['PREDICTED_B'] 
predicting_d = predicting_d.drop('PREDICTED_B', axis = 1) 
predicting_d

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,TARGET_B
0,11,5,10.0,7.741935,4,89,1,92,1
1,16,3,25.0,15.666667,2,94,1,93,0
2,14,5,5.0,7.481481,4,90,1,91,1
3,17,3,10.0,6.812500,4,87,1,87,1
4,15,17,15.0,6.864865,2,86,1,93,0
...,...,...,...,...,...,...,...,...,...
95407,18,10,25.0,25.000000,1,96,1,96,0
95408,16,1,20.0,20.000000,1,96,1,96,0
95409,17,14,10.0,8.285714,3,95,1,96,1
95410,16,6,18.0,12.146341,4,86,1,90,1


In [97]:
predictin_d_1 = predicting_d.loc[predicting_d['TARGET_B'] == 1].reset_index(drop=True)
predictin_d_1

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,TARGET_B
0,11,5,10.0,7.741935,4,89,1,92,1
1,14,5,5.0,7.481481,4,90,1,91,1
2,17,3,10.0,6.812500,4,87,1,87,1
3,14,3,11.0,7.642857,1,87,1,87,1
4,14,7,11.0,6.200000,3,94,1,95,1
...,...,...,...,...,...,...,...,...,...
37455,18,4,11.0,9.750000,3,95,1,95,1
37456,14,8,8.0,5.900000,4,94,1,94,1
37457,14,4,2.0,3.375000,4,90,1,96,1
37458,17,14,10.0,8.285714,3,95,1,96,1


FITTING IN THE TRANSFORMER AND IN THE MODEL TO GET THE PREDICTIONS

In [98]:
X1 = transformer.transform(predictin_d_1)

In [99]:
model3 = KNeighborsRegressor(n_neighbors=5)
model3.fit(X_train, y_train)
predictions = model3.predict(X1)
predictions

array([ 9.6,  6.8,  8.8, ...,  7.2,  9.2, 16.6])

In [100]:
predicted_D = pd.DataFrame(predictions, columns=['PREDICTED_D'])

In [101]:
predicted_D

,PREDICTED_D
0,9.6
1,6.8
2,8.8
3,8.8
4,12.2
...,...
37455,10.6
37456,8.6
37457,7.2
37458,9.2


In [105]:
predictions = pd.concat((predictin_d_1, predicted_D), axis=1)
predictions ['PREDICTED_B'] = predictions["TARGET_B"]
predictions = predictions.drop('TARGET_B', axis = 1) 
predictions

,CHILC2,HHD9,LASTGIFT,AVGGIFT,RFA_2F,ODATEW_YR,ODATEW_MM,MINRDATE_YR,PREDICTED_D,PREDICTED_B
0,11,5,10.0,7.741935,4,89,1,92,9.6,1
1,14,5,5.0,7.481481,4,90,1,91,6.8,1
2,17,3,10.0,6.812500,4,87,1,87,8.8,1
3,14,3,11.0,7.642857,1,87,1,87,8.8,1
4,14,7,11.0,6.200000,3,94,1,95,12.2,1
...,...,...,...,...,...,...,...,...,...,...
37455,18,4,11.0,9.750000,3,95,1,95,10.6,1
37456,14,8,8.0,5.900000,4,94,1,94,8.6,1
37457,14,4,2.0,3.375000,4,90,1,96,7.2,1
37458,17,14,10.0,8.285714,3,95,1,96,9.2,1
